In [1]:
import os
os.environ["OPENAI_API_KEY"] = "..."
os.environ["HUGGINGFACE_API_KEY"] = "..."

### Impact of Embeddings on Quality with Sub Question Query

In this tutorial, we load longer text (Fellowship of the Ring) and utilize Llama-Index Sub Question Query to evlauate a complex question around Frodo's character evolution.

In addition, we will iterate through different embeddings and chunk sizes and use TruLens to select the best one.

In [2]:
# NOTE: This is ONLY necessary in jupyter notebook.
# Details: Jupyter runs an event-loop behind the scenes. 
#          This results in nested event-loops when we start an event-loop to make async queries.
#          This is normally not allowed, we use nest_asyncio to allow it for convenience.  
import nest_asyncio
nest_asyncio.apply()

In [3]:
# Import main tools for building app
from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext, ResponseSynthesizer
from llama_index.indices.document_summary import DocumentSummaryIndexRetriever
from llama_index.tools import QueryEngineTool, ToolMetadata
from llama_index.query_engine import SubQuestionQueryEngine, RetrieverQueryEngine

# load data
alice = SimpleDirectoryReader(input_dir="./data/alice").load_data()

In [4]:
# Imports main tools for eval
from trulens_eval import TruLlama, Feedback, Tru, feedback
tru = Tru()

#hugs = feedback.Huggingface()
openai = feedback.OpenAI()

# Question/answer relevance between overall question and answer.
model_agreement = Feedback(openai.model_agreement).on_input_output()

No .env found in /Users/jreini/Desktop/development/trulens/trulens_eval/examples/frameworks/llama_index or its parents. You may need to specify secret keys in another manner.


✅ In model_agreement, input prompt will be set to *.__record__.main_input or `Select.RecordInput` .
✅ In model_agreement, input response will be set to *.__record__.main_output or `Select.RecordOutput` .


In [7]:
# iterate through embeddings and chunk sizes, evaluating each response's agreement with chatgpt using TruLens
embeddings = ['text-embedding-ada-001','text-embedding-ada-002']
query_engine_types = ['VectorStoreIndex','SubQuestionQueryEngine']

service_context=512

for embedding in(embeddings):
    for query_engine_type in query_engine_types:

            # build index and query engine
            index = VectorStoreIndex.from_documents(alice)

            # create embedding-based query engine from index
            query_engine = index.as_query_engine(embed_model=embedding)

            if query_engine_type == 'SubQuestionQueryEngine':
                service_context = ServiceContext.from_defaults(chunk_size=512)
                # setup base query engine as tool
                query_engine_tools = [
                    QueryEngineTool(
                        query_engine=query_engine,
                        metadata=ToolMetadata(name='Alice in Wonderland', description='THE MILLENNIUM FULCRUM EDITION 3.0')
                    )
                ]
                query_engine = SubQuestionQueryEngine.from_defaults(query_engine_tools=query_engine_tools, service_context=service_context)
            else:
                pass         

            tc = TruLlama(app_id = f'{query_engine_type}_{embedding}', app = query_engine, feedbacks = [model_agreement])

            response = tc.query("Describe Alice's growth from meeting the White Rabbit to challenging the Queen of Hearts?")

✅ app VectorStoreIndex_text-embedding-ada-001 -> default.sqlite
✅ feedback def. feedback_definition_hash_514dbce4eb825a25632cdb0626e7200c -> default.sqlite
✅ record record_hash_e3b485d9b03230306bd988ab6e20a5c8 from VectorStoreIndex_text-embedding-ada-001 -> default.sqlite
✅ app SubQuestionQueryEngine_text-embedding-ada-001 -> default.sqlite
✅ feedback def. feedback_definition_hash_514dbce4eb825a25632cdb0626e7200c -> default.sqlite
DEBUG
 
You will continually start seeing responses to the prompt:

Describe Alice's growth from meeting the White Rabbit to challenging the Queen of Hearts?

The right answer is:


Alice's growth from meeting the White Rabbit to challenging the Queen of Hearts is marked by her increasing confidence and independence. When she first meets the White Rabbit, she is timid and unsure of herself, asking for help and not sure of her identity. As she progresses through the story, she begins to take charge of her own destiny, questioning the Queen and standing up for 

In [8]:
tru.run_dashboard()

✅ record record_hash_3f639fc1b2ffae1c1feeba6cd768090c from SubQuestionQueryEngine_text-embedding-ada-002 -> default.sqlite
Starting dashboard ...


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

Dashboard started at http://192.168.50.34:8502 .


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>